In [1]:
import pandas as pd
import requests
import os
import json

## Get Registration data for EV 

In [6]:

CURRENT_PATH = os.getcwd()

In [5]:
DATA_PATH = os.path.join(CURRENT_PATH,"data")

In [7]:
reg_df = pd.DataFrame()

In [9]:
ev_reg_files = ['co_ev_registrations_public.xlsx',
 'ct_ev_registrations.xlsx',
 'fl_ev_registrations.xlsx',
 'mi_ev_registrations_public.xlsx',
 'mn_ev_registrations_public.xlsx',
 'nj_ev_registrations_public.xlsx',
 'ny_ev_registrations_public.xlsx',
 'or_ev_registrations_public.xlsx',
 'tx_ev_registrations_public.xlsx',
 'va_ev_registrations_public.xlsx',
 'vt_ev_registrations_public.xlsx',
 'wa_ev_registrations_public.xlsx',
 'wi_ev_registrations_public.xlsx']


In [26]:
for f in ev_reg_files:
    file_path = os.path.join(DATA_PATH,f)
    temp_df = pd.read_excel(file_path)
    temp_df['file_name'] = f
#     temp_df['state'] = file_path.split("_")[-4].split("\\")[-1] ## getting the name of the state from the file name
    reg_df = pd.concat([reg_df,temp_df])

C:\Users\SMaharana\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [38]:
reg_df['file_name'].value_counts()

ny_ev_registrations_public.xlsx    956830
wa_ev_registrations_public.xlsx    429192
nj_ev_registrations_public.xlsx     91143
or_ev_registrations_public.xlsx     76342
tx_ev_registrations_public.xlsx     74143
fl_ev_registrations.xlsx            67209
va_ev_registrations_public.xlsx     28039
mi_ev_registrations_public.xlsx     27518
co_ev_registrations_public.xlsx     22751
mn_ev_registrations_public.xlsx     16177
ct_ev_registrations.xlsx            10569
wi_ev_registrations_public.xlsx      6380
vt_ev_registrations_public.xlsx      6196
Name: file_name, dtype: int64

In [40]:
reg_df['state'] = reg_df['file_name'].apply(lambda x: x.split("_")[0])

In [43]:
reg_df.head()

,County,County GEOID,DMV ID,Registration Expiration Date,Registration Valid Date,VIN Model Year,VIN Prefix,Vehicle Name,ZIP Code,file_name,state
0,NaN,NaN,1,2020-08-31 00:00:00,2019-08-07,K,5YJ3E1EA,NaN,80003,co_ev_registrations_public.xlsx,co
1,NaN,NaN,1,2020-08-31 00:00:00,2019-08-07,K,5YJ3E1EA,NaN,80241,co_ev_registrations_public.xlsx,co
2,NaN,NaN,1,2020-08-31 00:00:00,2019-08-07,G,1N4BZ0CP,NaN,80212,co_ev_registrations_public.xlsx,co
3,NaN,NaN,1,2020-08-31 00:00:00,2019-08-07,E,5YJSA1H1,NaN,80210,co_ev_registrations_public.xlsx,co
4,NaN,NaN,1,2020-07-31 00:00:00,2019-08-07,C,5YJSA1CN,NaN,80304,co_ev_registrations_public.xlsx,co


In [45]:
reg_df['VIN Prefix'].nunique()

314

### Saving all data in a csv [Can directly import from here]

In [42]:
reg_df.to_csv('data/registration_data_all_states.csv')

### Getting aggregated data by zipcodes

I am still not able to figure out the granularity in the data sets.
I tried to compare the number of rows with the BI dashboard, but it seems like we need to take the aggregate for the latest registration date.

So, I will get the count of EVs for the latest registration date per Zip Code

In [48]:
unique_state_list = reg_df['state'].unique().tolist()


For each state
    - Get max registration date
    - Get count of rows by ZIP code



In [52]:
zip_df = pd.DataFrame()

In [77]:
for state in unique_state_list:
    print ("processing..", state)
    try:
        ## filter dataframe by state
        df = reg_df[reg_df['state'] ==state]
        ## get max reg date for each state
        max_reg_date = df['Registration Valid Date'].max()
        ## get count by zip code
        temp_df = pd.DataFrame({'count' : df.groupby(['ZIP Code']).size()}).reset_index()
        temp_df['state'] = state
        zip_df = pd.concat([zip_df,temp_df])
    except:
        print ("Skipping..", state)
    

processing.. co
processing.. ct
processing.. fl
Skipping.. fl
processing.. mi
processing.. mn
processing.. nj
processing.. ny
processing.. or
processing.. tx
processing.. va
Skipping.. va
processing.. vt
processing.. wa
processing.. wi


In [84]:
zip_df['ZIP Code'].nunique()

8923

8923 zip codes processed and some states(fl, va) have no zip codes

In [86]:
zip_df.to_csv('data/zip_data.csv')